In [ ]:
# Vamos a realizar una prueba solo son la prediccion de latitudes
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.regularizers import l2
import sys
sys.path.append('../')
from Utils import utils_nn as utlnn


In [ ]:
x_test = pd.read_excel("../Train_Test/Dataset_Separado/x_test.xlsx")
x_train = pd.read_excel("../Train_Test/Dataset_Separado/x_train.xlsx")
y_test = pd.read_excel("../Train_Test/Dataset_Separado/y_test.xlsx")
y_train = pd.read_excel("../Train_Test/Dataset_Separado/y_train.xlsx")




In [ ]:
out_x_coord = [f'x_{i}' for i in range(1,101)]
y_train_x=y_train[out_x_coord]
y_test_x = y_test[out_x_coord]



In [ ]:
epoch = 1200
b_s = 70

while epoch <= 1500:
  while b_s <= 130:
    l2_reg = 0.00
    inputs = Input(shape=(9,))
    encoded = Dense(9, activation='relu',kernel_regularizer=l2(l2_reg))(inputs)  # Compresión 8 V3 16 n 
    decoded = Dense(16, activation='relu',kernel_regularizer=l2(l2_reg))(encoded)
    encoded = Dense(32, activation='relu',kernel_regularizer=l2(l2_reg))(encoded)
    encoded = Dense(64, activation = 'relu',kernel_regularizer=l2(l2_reg))(encoded)
    encoded = Dense(80, activation = 'relu',kernel_regularizer=l2(l2_reg))(encoded)
    encoded = Dense(90, activation = 'relu',kernel_regularizer=l2(l2_reg))(encoded)
    decoded = Dense(100, activation='linear',kernel_regularizer=l2(l2_reg), name = 'x_output')(encoded)  # Reconstrucción a 100 puntos
    autoencoder_x = Model(inputs, decoded)
    autoencoder_x.compile(optimizer = 'adam', loss='mse')
    autoencoder_x.summary()

    history = autoencoder_x.fit(x_train,y_train_x,
                          epochs = epoch,
                          batch_size = b_s, 
                          validation_split = 0.1)

    loss = autoencoder_x.evaluate(x_test, y_test_x)
    
    if loss <= 35:
      autoencoder_x.save(f'../modelos_entrenamiento/modelos_x/mod_x_{epoch}_{b_s}_vs10_Adam_loss_{round(loss)}.keras')
    print(f'Pérdida en datos de Test: {loss} epoch: {epoch}, batch_size: {b_s}')
    b_s += 20  
  b_s = 80
  epoch +=40

In [ ]:

l2_reg = 0.00
inputs = Input(shape=(9,))

encoded = Dense(9, activation='relu',kernel_regularizer=l2(l2_reg))(inputs)  # Compresión 8 V3 16 n 
decoded = Dense(16, activation='relu',kernel_regularizer=l2(l2_reg))(encoded) #V3
encoded = Dense(32, activation='relu',kernel_regularizer=l2(l2_reg))(encoded)
encoded = Dense(64, activation = 'relu',kernel_regularizer=l2(l2_reg))(encoded)
encoded = Dense(80, activation = 'relu',kernel_regularizer=l2(l2_reg))(encoded)
encoded = Dense(90, activation = 'relu',kernel_regularizer=l2(l2_reg))(encoded)

decoded = Dense(100, activation='linear',kernel_regularizer=l2(l2_reg), name = 'x_output')(encoded)  # Reconstrucción a 100 puntos

autoencoder_x = Model(inputs, decoded)
autoencoder_x.compile(optimizer = 'adam', loss='mse')
autoencoder_x.summary()

In [ ]:
history = autoencoder_x.fit(x_train,y_train_x,
                          epochs = 900,
                          batch_size = 90, 
                          validation_split = 0.1) 

In [ ]:
loss = autoencoder_x.evaluate(x_test, y_test_x)
print(f'Pérdida en datos de Test: {loss}')

In [ ]:
if loss <100:
  print("True")
else :
  print(round(loss))

In [ ]:
idx = 377

 # Elegir una muestra para comparar

# Predicción de una muestra 
y_pred = autoencoder_x.predict(np.expand_dims(x_test.iloc[idx], axis=0))
###############################################
y_true = y_test_x.iloc[idx] # Se obtine Algo de tipo Serie
y_true=y_true.to_numpy() # Transform a Numpy array
y_pred = y_pred.flatten() # [[...,...,...,....,]]

In [ ]:
##Graficamos 
# Pos x
plt.figure(figsize=(15, 6))
plt.plot(y_true, label='Posiciones X reales', linestyle='None', marker='.')
plt.plot(y_pred, label='Posiciones X predichas', linestyle = 'None',marker='o')
# plt.ylim(-35,-50) ##(-60,-30)
plt.title('Comparación de Posiciones X')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
ecm, recm = utlnn.calculate_ECM_RECM(y_true,y_pred)
print(f'Error Cuadratico Medio:{ecm}, Raiz del Error Cuadratico Medio:{recm} [Km]')


In [ ]:
save_model = False

if save_model == True:
  autoencoder_x.save('../modelos_entrenamiento/modelos_x/mod_x_800_160_vs10_AdamW_l2_0003.keras')

In [ ]:
# Pérdida en datos de Test: 30.48688316345215  / mod_x_800_130_vs10 (Despues de 55 pasadas)
# Pérdida en datos de Test: 112.4796371459961 / mod_x_700_80_vs10
# Pérdida en datos de Test: 93.09298706054688 / mod_x_800_90_vs10
# Pérdida en datos de Test: 36.03205490112305 / mod_x_800_160_vs10_AdamW_l2_0003